# MVVM + Hexagonal Architecture in SEVA (Beginner-Friendly)

This notebook explains the **MVVM + Hexagonal** structure used in this codebase. It is written for **university beginners** who have only written basic Python scripts and are now going to work on this project.

**Learning goals**
- Understand the *layers* (Views, ViewModels, UseCases, Adapters, Domain).
- See how **Ports** (interfaces) enable swapping adapters without changing the use case logic.
- Walk through a real workflow: **start experiment → submit via Job adapter**.
- Practice with **mock adapters** so you can experiment safely.
Follow-up: **Part 2** covers Views & ViewModels in detail and shows how they are initialized.


In [ ]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))


## 1) Quick map of the architecture

> Think of this as two ideas combined:
>
> **MVVM** keeps UI state separate from logic.  
> **Hexagonal** (Ports & Adapters) keeps external I/O separate from business logic.

```
UI Layer (Views)         <-- rendering only
    |
ViewModels               <-- UI state + commands
    |
UseCases                 <-- orchestration / workflow
    |
Ports (interfaces)       <-- contract boundary
    |
Adapters                 <-- I/O implementations (HTTP, filesystem, etc.)
    |
External services / devices
```

The SEVA code follows this **strict separation**:
- Views never call network/filesystem APIs.
- ViewModels never build raw HTTP payloads.
- UseCases never perform direct I/O.
- Adapters do all I/O and must match the Port contract.

## 2) Ports and Adapters (the Hexagonal core)

A **Port** is a contract (interface). An **Adapter** is a concrete implementation.

In this repo:
- Ports live in: `seva/domain/ports.py`
- Example port: `JobPort`
- Adapter implementations:
  - `JobRestAdapter` (real HTTP adapter) in `seva/adapters/job_rest.py`
  - `JobRestMock` (offline mock adapter) in `seva/adapters/job_rest_mock.py`

**Important idea:** if two adapters implement the same Port, the use case can swap between them with no changes.
The Port defines at least the **method names and signatures** (e.g., `start_batch`, `poll_group`).
Adapters can translate internally, but they must provide these names so use cases can call them.


## 3) The workflow we will simulate

We will simulate a **start experiment** flow using a mock adapter:

1. Build a typed `ExperimentPlan` in a UseCase (`BuildExperimentPlan`).
2. Submit it through a Port (`JobPort.start_batch`).
3. Read results from the same port using a mock adapter.

This is exactly how the real application works — just without real HTTP.

In [ ]:
# Imports from the repo
from seva.usecases.build_experiment_plan import (
    BuildExperimentPlan,
    ExperimentPlanRequest,
    ModeSnapshot,
    WellSnapshot,
)
from seva.usecases.start_experiment_batch import StartExperimentBatch
from seva.adapters.job_rest_mock import JobRestMock

### Build an ExperimentPlan (UseCase)

The UI never creates raw JSON directly. Instead, it provides **snapshots**, and the
UseCase builds a typed `ExperimentPlan` domain object.

In [ ]:
request = ExperimentPlanRequest(
    experiment_name="Intro Experiment",
    subdir="training",
    client_datetime_override=None,
    wells=("A1", "A2"),
    well_snapshots=(
        WellSnapshot(
            well_id="A1",
            modes=(
                ModeSnapshot(
                    name="CV",
                    params={
                        "cv.start_v": -0.2,
                        "cv.vertex1_v": 0.3,
                        "cv.vertex2_v": -0.3,
                        "cv.final_v": 0.0,
                        "cv.scan_rate_v_s": 0.1,
                        "cv.cycles": 2,
                    },
                ),
            ),
        ),
        WellSnapshot(
            well_id="A2",
            modes=(
                ModeSnapshot(
                    name="CV",
                    params={
                        "cv.start_v": -0.1,
                        "cv.vertex1_v": 0.2,
                        "cv.vertex2_v": -0.2,
                        "cv.final_v": 0.0,
                        "cv.scan_rate_v_s": 0.05,
                        "cv.cycles": 1,
                    },
                ),
            ),
        ),
    ),
)

build_plan = BuildExperimentPlan()
plan = build_plan(request)
plan

### Simulating progress with a timed mock

To visualize progress changes, we can extend the mock to return a `progress_pct` that
moves from 0 to 100 over ~1 minute.


In [ ]:
import time


class TimedJobMock(JobRestMock):
    def __post_init__(self):
        super().__post_init__()
        self._group_start = {}

    def start_batch(self, plan):
        group_id, runs = super().start_batch(plan)
        self._group_start[group_id] = time.time()
        return group_id, runs

    def poll_group(self, run_group_id):
        snapshot = super().poll_group(run_group_id)
        started = self._group_start.get(run_group_id, time.time())
        elapsed = max(0.0, time.time() - started)
        progress = min(100.0, (elapsed / 60.0) * 100.0)
        for box in snapshot.get("boxes", {}).values():
            for run in box.get("runs", []):
                run["progress_pct"] = progress
        return snapshot


### Submit via a Port (using a mock adapter)

Now we use **StartExperimentBatch**, which depends only on the `JobPort` interface.
We inject the **mock adapter** so there is no real HTTP.
You can also inject the AMETEK adapter here (see the example below) because it
implements the same `JobPort` method names.


In [ ]:
job_port = TimedJobMock()  # Adapter implements JobPort
start_batch = StartExperimentBatch(job_port=job_port)
result = start_batch(plan)
result


### Poll status (still through the Port)

The mock adapter stores state in memory, so we can query it just like the real API.
Run the polling cell multiple times (or press "Run" again) to see the progress change.


In [ ]:
run_group_id = result.run_group_id
snapshot = job_port.poll_group(run_group_id)
snapshot


## 4) Swapping adapters (same Port, new implementation)

Here is the key: **UseCases do not care** *which* adapter they receive, as long as it
implements the Port contract.

If you swap this:

```python
job_port = JobRestMock()
```

for this:

```python
from seva.adapters.job_rest import JobRestAdapter
from seva.domain.entities import BoxId

job_port = JobRestAdapter(
    base_urls={BoxId("box-1"): "http://localhost:8000"},
    api_keys={BoxId("box-1"): "my-key"},
)
```

Then the **same use case** (`StartExperimentBatch`) works with a real backend.

### Example: swapping to an AMETEK adapter
Imagine a legacy AMETEK API that builds experiments differently and returns different
polling shapes. The **JobPort contract** is still the same, so the adapter
translates between AMETEK and our domain objects.

Tip: you can point `base_urls` to the **Raspberry Pi server IP** if that is hosting the API.


In [ ]:
from seva.domain.ports import JobPort
from seva.domain.entities import ExperimentPlan


class AmetekJobAdapter(JobPort):
    """Example adapter that translates to a legacy AMETEK API.

    This mock-style adapter just prints to show where translation happens.
    """

    def start_batch(self, plan: ExperimentPlan):
        print("AMETEK: translate plan -> API payload")
        # TODO: build AMETEK-specific payload here
        return ("ametek-group-1", {})

    def cancel_run(self, box_id, run_id):
        print("AMETEK: cancel run", box_id, run_id)

    def cancel_runs(self, box_to_run_ids):
        print("AMETEK: cancel runs", box_to_run_ids)

    def cancel_group(self, run_group_id):
        print("AMETEK: cancel group", run_group_id)

    def poll_group(self, run_group_id):
        print("AMETEK: poll status for", run_group_id)
        # TODO: translate AMETEK status -> normalized snapshot
        return {"boxes": {}, "wells": []}

    def download_group_zip(self, run_group_id, target_dir):
        print("AMETEK: download artifacts", run_group_id, target_dir)
        return target_dir


## 5) Where MVVM fits in this workflow

- **View**: draws UI and sends user actions (no I/O, no domain logic).
- **ViewModel**: collects user inputs, builds request snapshots, calls use cases.
- **UseCase**: builds plans and orchestrates workflows (`BuildExperimentPlan`, `StartExperimentBatch`).
- **Port**: contract for I/O (`JobPort`).
- **Adapter**: real or mock I/O (`JobRestAdapter`, `JobRestMock`).

This is why the **core logic stays testable**: you can run most of the system using mocks.

## 6) What to explore next

Try these exercises:
1. Add a **third well** to the request and see how `JobRestMock` returns more runs.
2. Modify the `ModeSnapshot` to use a different mode (e.g., `EIS`) and inspect failures.
3. Create your **own** mock adapter that implements the same `JobPort` methods.
4. Read the `JobPort` contract in `seva/domain/ports.py` and map every method to where it is used.